In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
tf.logging.set_verbosity(tf.logging.ERROR) # 警告を出さないようにする

In [2]:
# MNISTをダウンロードしてテスト用データの参照情報を変数に格納
mnist = input_data.read_data_sets("data/", one_hot=True)
test_images = mnist.test.images   # テスト用の画像を変数に格納
test_labels = mnist.test.labels   # テスト用の正解ラベルを変数に格納

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz


In [3]:
# 学習率、エポック数、ミニバッチのサイズ、非ドロップアウト率を設定
learning_rate = 0.001  # 学習率
training_epochs = 1500 # エポック数
batch_size = 100       # ミニバッチのサイズ
dropout = 0.5          # 非ドロップアウト率

In [4]:
# データフローグラフ

with tf.name_scope("input"):
    '''
    入力層（ニューロン数＝784→1に相当）
    '''
    # 入力データ（行数可変, 各データのサイズ＝784列）
    x = tf.placeholder(tf.float32, [None, 784])
    # 画像データの2次元配列「データ数×784」を
    # 「データ数×28×28×1」の4階テンソルに変換
    out_img = tf.reshape(x,[-1, 28, 28, 1])

with tf.name_scope("layer1_conv1"):
    '''
    （第1層）畳み込み層1
    ニューロン数：16
    出力：1ニューロンあたり(28, 28, 1)の3階テンソルを16個出力

    '''
    # 2次元フィルター（3×3を16枚）
    f1 = tf.Variable(
        tf.truncated_normal([3,         # フィルターの縦サイズ
                             3,         # フィルターの横サイズ
                             1,         # 入力画像のチャンネル数
                             16],       # フィルターの枚数
                            stddev=0.1))
    # 畳み込みを行うopノード
    conv1 = tf.nn.conv2d(out_img,       # 28×28の画像
                         f1,            # 2次元フィルター
                         strides=[1,    # 入力データを1ずつ順送り
                                  1,    # 縦方向のストライドは1
                                  1,    # 横方向のストライドは1
                                  1],   # チャンネルのストライドは1
                         padding='SAME' # パディングを行う
                        )
    # フィルターのバイアス
    b1 = tf.Variable(
        tf.constant(0.1,                # 0.1で初期化
                    shape=[16]          # フィルターの数だけ用意
                    ))
    # 活性化関数はReLU
    # 出力：(データの数, 28（タテ）, 28（ヨコ）, 1（チャンネル）)×16
    out_conv1 = tf.nn.relu(conv1 + b1)
    
with tf.name_scope("layer2_conv2"):
    '''
    （第2層）畳み込み層2
    ニューロン数：32
    出力：1ニューロンあたり(28, 28, 1)の3階テンソルを32個出力

    '''
    # 2次元フィルター（3×3を32枚）
    f2 = tf.Variable(
        tf.truncated_normal([3,         # フィルターの縦サイズ
                             3,         # フィルターの横サイズ
                             16,        # 入力画像のチャンネル数
                             32],       # フィルターの枚数
                            stddev=0.1))
    # 畳み込みを行うopノード
    conv2 = tf.nn.conv2d(out_conv1,     # 28×28の画像
                         f2,            # 2次元フィルター
                         strides=[1,    # 入力データを1ずつ順送り
                                  1,    # 縦方向のストライドは1
                                  1,    # 横方向のストライドは1
                                  1],   # チャンネルのストライドは1
                         padding='SAME' # パディングを行う
                        )
    # フィルターのバイアス
    b2 = tf.Variable(
        tf.constant(0.1,                # 0.1で初期化
                    shape=[32]          # フィルターの数だけ用意
                    ))
    # 活性化関数はReLU
    # 出力：(画像の枚数, 28（タテ）, 28（ヨコ）, 1（チャンネル）)×32
    out_conv2 = tf.nn.relu(conv2 + b2)
    
with tf.name_scope("layer3_pooling1"):
    '''
    （第3層）プーリング層1
    ユニット数：32
    出力：1ユニットあたり(14, 14, 1)の3階テンソルを32個出力
    
    '''
    out_pool1 = tf.nn.max_pool(
        out_conv2,     # 28×28の画像
        ksize=[1,      # データに対するウィンドウサイズ
               2,      # タテ方向のウィンドウサイズ
               2,      # ヨコ方向のウィンドウサイズ
               1       # チャンネル方向のウィンドウサイズ
              ],
        strides=[1,    # データのストライド量
                 2,    # タテ方向のストライド量
                 2,    # ヨコ方向のストライド量
                 1     # チャンネル方向のストライド量
                ],
        padding='SAME' # フィルターに対するパディングを行う
    )
    
with tf.name_scope("layer4_conv3"):
    '''
    （第4層）畳み込み層3
    ニューロン数：64
    出力：1ニューロンあたり(14, 14, 1)の3階テンソルを64個出力

    '''
    # 2次元フィルター（3×3を64枚）
    f3 = tf.Variable(
        tf.truncated_normal([3,          # フィルターの縦サイズ
                             3,          # フィルターの横サイズ
                             32,         # 画像のチャンネル数
                             64],        # フィルターの枚数
                            stddev=0.1))
    # 畳み込みを行うopノード
    conv3 = tf.nn.conv2d(out_pool1,      # 14×14の画像
                         f3,             # 2次元フィルター
                         strides=[1,     # 入力データを1ずつストライド
                                  1,     # 縦方向のストライドは1
                                  1,     # 横方向のストライドは1
                                  1],    # チャンネルのストライドは1
                         padding='SAME')
    # フィルターのバイアス
    b3 = tf.Variable(
        tf.constant(0.1,                 # 0.1で初期化
                    shape=[64]           # フィルターの数だけ用意
                   ))
    # 活性化関数はReLU
    # 出力：(像の枚数, 14（タテ）,14（ヨコ）,64（チャンネル）)
    out_conv3 = tf.nn.relu(conv3 + b3)
    
with tf.name_scope("layer5_pooling2"):
    '''
    （第5層）プーリング層1
    ユニット数：64
    出力：1ユニットあたり(7, 7, 1)の3階テンソルを64個出力
    
    '''
    out_pool2 = tf.nn.max_pool(
        out_conv1,
        ksize=[1,      # データに対するウィンドウサイズ
               2,      # タテ方向のウィンドウサイズ
               2,      # ヨコ方向のウィンドウサイズ
               1       # チャンネル方向のウィンドウサイズ
              ],
        strides=[1,    # データのストライド量
                 2,    # タテ方向のストライド量
                 2,    # ヨコ方向のストライド量
                 1     # チャンネル方向のストライド量
                ],
        padding='SAME' # フィルターに対するパディングを行う
    )

with tf.name_scope("dropout"):
    '''
    ドロップアウト層
    出力：1ユニットあたり(7, 7, 1)の3階テンソルを64個出力

    '''
    # 非ドロップアウト率を保持するプレースホルダー
    keep_prob = tf.placeholder(tf.float32)
    # ドロップアウトを行うopノード
    out_drop = tf.nn.dropout(out_pool2, keep_prob)

with tf.name_scope("flatten"):
    '''
    Flatten（ニューロン数＝7×7×64）
    (画像の枚数, 7（タテ）,7（ヨコ）,64（チャンネル）)を
    (画像の枚数, 7×7×64=3136)の2階テンソルに変換

    '''
    out_flat = tf.reshape(out_drop,
                              [-1, 7*7*64] # (画像の枚数, 画像データ)
                             )

with tf.name_scope("layer6_binding"):
    '''
    （第6層）全結合層
    ニューロン数：128
    出力：要素数(128)の1階テンソルを出力

    '''
    # 重み(前層のニューロン数＝7×7×64行, ニューロン数＝128列)
    w_comb1 = tf.Variable(tf.truncated_normal([7*7*64, 128], stddev=0.1))
    # バイアス(ニューロン数＝128列)
    b_comb1 = tf.Variable(tf.constant(0.1, shape=[128]))
    # 活性化関数:出力((前層の出力行数＝画像の枚数行, ニューロン数＝128列)
    out_comb1 = tf.nn.relu(tf.matmul(out_flat, w_comb1) + b_comb1)

with tf.name_scope("layer7_output"):
    '''
    （第7層）出力層
    ニューロン数：10
    出力：要素数(10)の1階テンソルを出力

    '''
    # 重み(前層のニューロン数＝128行,クラス数＝ニューロン数＝10列)
    w_comb2 = tf.Variable(tf.truncated_normal([128, 10], stddev=0.1))
    # バイアス(クラス数＝ニューロン数＝10列)
    b_comb2 = tf.Variable(tf.constant(0.1, shape=[10]))
    # 活性化関数:出力((前層の出力行数＝画像の枚数行, ニューロン数＝10列)
    out = tf.nn.softmax(tf.matmul(out_comb1, w_comb2) + b_comb2)
    
#正解データの型を定義
#誤差関数（クロスエントロピー）
with tf.name_scope("loss"):
    # 正解ラベルを保持するプレースホルダー(行数可変, 列数は正解ラベルと同じ1)
    t = tf.placeholder(tf.float32, [None, 10])
    # クロスエントロピー誤差
    loss = tf.reduce_mean(
        -tf.reduce_sum(t*tf.log(out + 1e-5), axis=[1])
        )

# 訓練(ミニマイザー)
with tf.name_scope("train"):
    train_step = tf.train.AdamOptimizer(
        learning_rate).minimize(loss)

# 評価
with tf.name_scope("accuracy"):
    # (out＝t)の最大値を比較
    correct = tf.equal(tf.argmax(out,1), tf.argmax(t,1))
    # True(正解＝1)の平均を取る
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

# 変数を初期化するopノード
init =tf.global_variables_initializer()

In [5]:
# セッション部
with tf.Session() as sess:
    sess.run(init)   # initノードを実行して変数を初期化

    for step in range(training_epochs):
        # ミニバッチを抽出
        train_images, train_labels = mnist.train.next_batch(batch_size)
        sess.run(
            train_step,                    # ミニマイザーを実行
            feed_dict={x:train_images,     # 訓練データのミニバッチ
                       t:train_labels,     # 正解ラベルのミニバッチ
                       keep_prob: dropout} # ドロップアウト
            )

        # 100回ごとにテストデータを使用して精度を出力
        step += 1
        if step % 100 == 0:
            acc_val = sess.run(
                accuracy,                  # クロスエントロピー誤差の平均オペレーション
                feed_dict={x:test_images,  # テストテータをセット
                           t:test_labels,  # テストデータの正解ラベル
                           keep_prob: 1.0} # 非ドロップアウト率を1.0にする
            )
            print('Step %d: accuracy = %.2f' % (step, acc_val))

Step 100: accuracy = 0.88
Step 200: accuracy = 0.90
Step 300: accuracy = 0.92
Step 400: accuracy = 0.94
Step 500: accuracy = 0.95
Step 600: accuracy = 0.96
Step 700: accuracy = 0.96
Step 800: accuracy = 0.96
Step 900: accuracy = 0.97
Step 1000: accuracy = 0.97
Step 1100: accuracy = 0.97
Step 1200: accuracy = 0.97
Step 1300: accuracy = 0.97
Step 1400: accuracy = 0.98
Step 1500: accuracy = 0.97
